In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import class_weight, shuffle

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import ProjectedGradientDescent

warnings.filterwarnings('ignore')
print("All libraries imported successfully.")

2025-11-01 14:18:37.091504: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/reshma/Documents/Coding/capstone_project/attack/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/reshma/Documents/Coding/capstone_project/attack/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/reshma/Documents/Coding/capstone_project/attack/lib/python3.9/site-packages/art/estimators/cert

All libraries imported successfully.


In [ ]:
DATA_PATH = 'CIC-Darknet2020.csv'
TARGET_LABELS = ['Tor', 'Non-Tor', 'VPN', 'NonVPN']

BASELINE_MODEL_PATH = 'model-multi.h5'
SCALER_PATH = 'scaler-multi.pkl'
DEFENDED_MODEL_PATH = 'model-defended-v2.h5'

ADV_DATA_PATH = 'X_train_adv-v2.npy'
ADV_LABELS_PATH = 'y_train_adv_labels_ohe-v2.npy'
ADV_LABELS_ENCODED_PATH = 'y_train_adv_labels_encoded-v2.npy'

PGD_EPS = 0.1
PGD_EPS_STEP = 0.01
PGD_MAX_ITER = 40
PGD_BATCH_SIZE = 64

In [ ]:
def load_all_data():
    try:
        df = pd.read_csv(DATA_PATH)
    except FileNotFoundError:
        print(f"Error: '{DATA_PATH}' not found.")
        return None

    df.columns = [*df.columns[:-2], 'Label', 'Label_Type']
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    df_multi = df[df['Label'].isin(TARGET_LABELS)].copy()

    non_feature_cols = ['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Timestamp', 'Label', 'Label_Type']
    X = df_multi.drop(columns=non_feature_cols).apply(pd.to_numeric)
    y = df_multi['Label']

    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    n_classes = len(le.classes_)
    target_names = le.classes_
    y_ohe = to_categorical(y_encoded, num_classes=n_classes)
    
    print("--- Class Encoding Mapping ---")
    for index, label in enumerate(le.classes_):
        print(f"Class Index {index} -> {label}")

    X_train, X_test, y_train_ohe, y_test_ohe, y_train_encoded, y_test_encoded = train_test_split(
        X, y_ohe, y_encoded,
        test_size=0.2,
        random_state=42,
        stratify=y_encoded
    )

    try:
        scaler = joblib.load(SCALER_PATH)
        print(f"\nScaler '{SCALER_PATH}' loaded successfully.")
    except FileNotFoundError:
        print(f"Error: '{SCALER_PATH}' not found. Did you run the baseline notebook?")
        return None
        
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # --- Reshape for CNN ---
    n_features = X_test_scaled.shape[1]
    X_train_cnn = X_train_scaled.reshape((X_train_scaled.shape[0], n_features, 1))
    X_test_cnn = X_test_scaled.reshape((X_test_scaled.shape[0], n_features, 1))
    
    print(f"Data preparation complete. Found {n_features} features and {n_classes} classes.")
    
    return X_train_cnn, y_train_ohe, y_train_encoded, X_test_cnn, y_test_ohe, y_test_encoded, target_names, n_features, n_classes

In [ ]:
def get_art_classifier_multi(model_path, n_features, n_classes):
    try:
        model = load_model(model_path)
        print(f"Model '{model_path}' loaded successfully.")
    except Exception as e:
        print(f"Error: Model file '{model_path}' not found or failed to load.")
        print(f"Details: {e}")
        return None, None
        
    loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    classifier = TensorFlowV2Classifier(
        model=model,
        loss_object=loss_object,
        input_shape=(n_features, 1),
        nb_classes=n_classes,
        channels_first=False
    )
    return model, classifier

def create_multi_class_cnn(input_shape, n_classes):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape, padding='same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(128, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(256, kernel_size=3, activation='relu', padding='same'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(n_classes, activation='softmax'))
    
    model.compile(optimizer=Adam(learning_rate=0.0005),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    print("New, un-trained CNN model created.")
    return model

In [ ]:
data = load_all_data()

if data:
    X_train_cnn, y_train_ohe, y_train_encoded, \
    X_test_cnn, y_test_ohe, y_test_encoded, \
    target_names, n_features, n_classes = data
    
    X_train_art = X_train_cnn.astype(np.float32)
    
    print(f"\nLoading baseline model '{BASELINE_MODEL_PATH}' to create attacks...")
    baseline_model, classifier = get_art_classifier_multi(BASELINE_MODEL_PATH, n_features, n_classes)
else:
    print("Data loading failed. Cannot proceed.")

--- Class Encoding Mapping ---
Class Index 0 -> Non-Tor
Class Index 1 -> NonVPN
Class Index 2 -> Tor
Class Index 3 -> VPN

Scaler 'scaler-multi.pkl' loaded successfully.
Data preparation complete. Found 76 features and 4 classes.

Loading baseline model 'model-multi.h5' to create attacks...


Model 'model-multi.h5' loaded successfully.


In [ ]:
if os.path.exists(ADV_DATA_PATH) and os.path.exists(ADV_LABELS_PATH):
    print("\n" + "="*50)
    print(f"STEP 1: Found existing adversarial data. Loading files...")
    X_train_adv = np.load(ADV_DATA_PATH)
    y_train_ohe_adv = np.load(ADV_LABELS_PATH)
    y_train_encoded_adv = np.load(ADV_LABELS_ENCODED_PATH)
    print("Adversarial training data loaded from local files.")
    
else:
    print("\n" + "="*50)
    print(f"STEP 1: No existing data found. Generating STRONGER PGD attack...")
    print(f"This will be slow (max_iter=40). Attacking {len(X_train_art)} samples...")
    print("="*50 + "\n")

    if 'classifier' not in locals():
        print("Error: Classifier not loaded. Cannot generate attacks.")
    else:
        attack = ProjectedGradientDescent(
            classifier,
            eps=PGD_EPS,
            eps_step=PGD_EPS_STEP,
            max_iter=PGD_MAX_ITER,
            random_eps=True,
            batch_size=PGD_BATCH_SIZE,
            verbose=True
        )

        X_train_adv = attack.generate(x=X_train_art, y=y_train_ohe)
        
        print("\nAdversarial training data generated.")
        print(f"Saving data to: {ADV_DATA_PATH}")
        np.save(ADV_DATA_PATH, X_train_adv)
        
        print(f"Saving labels to: {ADV_LABELS_PATH}")
        np.save(ADV_LABELS_PATH, y_train_ohe)
        y_train_ohe_adv = y_train_ohe
        
        print(f"Saving encoded labels to: {ADV_LABELS_ENCODED_PATH}")
        np.save(ADV_LABELS_ENCODED_PATH, y_train_encoded)
        y_train_encoded_adv = y_train_encoded


STEP 1: No existing data found. Generating STRONGER PGD attack...
This will be slow (max_iter=40). Attacking 126852 samples...



PGD - Batches: 1983it [1:16:42,  2.03s/it]2025-11-01 15:35:46.582298: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence



Adversarial training data generated.
Saving data to: X_train_adv-v2.npy
Saving labels to: y_train_adv_labels_ohe-v2.npy
Saving encoded labels to: y_train_adv_labels_encoded-v2.npy


In [ ]:
if 'X_train_adv' in locals():
    print("\n" + "="*50)
    print("STEP 2: Creating and shuffling hardened dataset...")
    print("="*50 + "\n")
    
    X_hardened = np.concatenate((X_train_art, X_train_adv), axis=0)
    y_hardened_ohe = np.concatenate((y_train_ohe, y_train_ohe_adv), axis=0)
    y_hardened_encoded = np.concatenate((y_train_encoded, y_train_encoded_adv), axis=0)
    
    print(f"Original training data shape: {X_train_art.shape}")
    print(f"New hardened training data shape: {X_hardened.shape}")
    print(f"New hardened labels shape: {y_hardened_ohe.shape}")

    X_hardened, y_hardened_ohe, y_hardened_encoded = shuffle(
        X_hardened, y_hardened_ohe, y_hardened_encoded, random_state=42
    )
    
    print("\nHardened dataset created and shuffled.")

else:
    print("Adversarial training data not found. Skipping dataset creation.")


STEP 2: Creating and shuffling hardened dataset...

Original training data shape: (126852, 76, 1)
New hardened training data shape: (253704, 76, 1)
New hardened labels shape: (253704, 4)

Hardened dataset created and shuffled.


In [ ]:
if 'X_hardened' in locals():
    print("\n" + "="*50)
    print("STEP 3: Training the new DEFENDED model...")
    print("This will take approx. 2x as long as the original training.")
    print("="*50 + "\n")

    input_shape = (n_features, 1)
    model_defended = create_multi_class_cnn(input_shape, n_classes)

    class_weights = class_weight.compute_class_weight(
        'balanced',
        classes=np.unique(y_hardened_encoded),
        y=y_hardened_encoded
    )
    class_weights_dict = dict(enumerate(class_weights))
    
    print("\nClass weights for new dataset:")
    for i in range(n_classes):
          print(f"Weight for class {i} ({target_names[i]}): {class_weights_dict[i]:.2f}")

    early_stopping = EarlyStopping(monitor='val_accuracy',
                                 patience=10,
                                 mode='max',
                                 restore_best_weights=True)
    
    history = model_defended.fit(
        X_hardened, y_hardened_ohe,
        epochs=50,
        batch_size=64,
        validation_data=(X_test_cnn, y_test_ohe), # Validate on clean test data
        callbacks=[early_stopping],
        class_weight=class_weights_dict,
        verbose=1
    )
    
    print("\nDefended model training complete.")

else:
    print("Hardened dataset not found. Skipping model training.")


STEP 3: Training the new DEFENDED model...
This will take approx. 2x as long as the original training.

New, un-trained CNN model created.

Class weights for new dataset:
Weight for class 0 (Non-Tor): 0.36
Weight for class 1 (NonVPN): 1.66
Weight for class 2 (Tor): 28.49
Weight for class 3 (VPN): 1.73
Epoch 1/50
3965/3965 ━━━━━━━━━━━━━━━━━━━━ 80s 19ms/step - accuracy: 0.6623 - loss: 0.8151 - val_accuracy: 0.8855 - val_loss: 0.4380
Epoch 2/50
3965/3965 ━━━━━━━━━━━━━━━━━━━━ 74s 19ms/step - accuracy: 0.9186 - loss: 0.3986 - val_accuracy: 0.9151 - val_loss: 0.2495
Epoch 3/50
3965/3965 ━━━━━━━━━━━━━━━━━━━━ 77s 19ms/step - accuracy: 0.9387 - loss: 0.3010 - val_accuracy: 0.9219 - val_loss: 0.2155
Epoch 4/50
3965/3965 ━━━━━━━━━━━━━━━━━━━━ 77s 19ms/step - accuracy: 0.9472 - loss: 0.2495 - val_accuracy: 0.9387 - val_loss: 0.1839
Epoch 5/50
3965/3965 ━━━━━━━━━━━━━━━━━━━━ 84s 21ms/step - accuracy: 0.9547 - loss: 0.2162 - val_accuracy: 0.9262 - val_loss: 0.2361
Epoch 6/50
3965/3965 ━━━━━━━━━━━━━━━

In [ ]:
if 'model_defended' in locals():
    print("\n" + "="*50)
    print("STEP 4: Saving and evaluating the defended model...")
    print("="*50 + "\n")

    model_defended.save(DEFENDED_MODEL_PATH)
    print(f"New defended model saved to '{DEFENDED_MODEL_PATH}'")

    print("\nEvaluating defended model on CLEAN test data:")
    loss, accuracy = model_defended.evaluate(X_test_cnn, y_test_ohe, verbose=0)
    
    print(f"Clean Test Accuracy (Defended): {accuracy * 100:.4f}%")
    print(f"Clean Test Loss (Defended): {loss:.4f}")

    y_pred_probs = model_defended.predict(X_test_cnn)
    y_pred_encoded = np.argmax(y_pred_probs, axis=1)

    print("\nClassification Report (Defended Model on Clean Data):")
    print(classification_report(y_test_encoded, y_pred_encoded, target_names=target_names))

else:
    print("Defended model not found. Skipping save/evaluation.")


STEP 4: Saving and evaluating the defended model...

New defended model saved to 'model-defended-v2.h5'

Evaluating defended model on CLEAN test data:
Clean Test Accuracy (Defended): 95.9135%
Clean Test Loss (Defended): 0.1122
992/992 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step

Classification Report (Defended Model on Clean Data):
              precision    recall  f1-score   support

     Non-Tor       1.00      0.99      1.00     22079
      NonVPN       0.88      0.86      0.87      4772
         Tor       0.90      0.87      0.89       279
         VPN       0.86      0.90      0.88      4584

    accuracy                           0.96     31714
   macro avg       0.91      0.91      0.91     31714
weighted avg       0.96      0.96      0.96     31714

